In [143]:
# import xml.etree.ElementTree as ET

# # fichier_xml = 'data/deft09_parlement_appr_fr.xml'
# fichier_xml = 'deft09_parlement_test_fr.xml'
# tree = ET.parse(fichier_xml)
# root = tree.getroot()

# list_contenu = []

# for texte_element in root.iter('texte'):
#     contenu = ET.tostring(texte_element, encoding='unicode', method='text').strip()
#     list_contenu.append(contenu)
# text_content = ' '.join(list_contenu)

# print(text_content)

# import xml.etree.ElementTree as ET
# import pandas as pd
# import random
# import csv

# tree = ET.parse(fichier_xml)
# root = tree.getroot()

# # Initialiser une liste pour stocker les données
# data = []

# # Parcourir les documents dans le fichier XML
# for doc in root.findall('doc'):
#     doc_id = doc.attrib['id']
    
#     # parti = doc.find('EVALUATION/EVAL_PARTI/PARTI').attrib['valeur']
    
#     texte = doc.find('texte/p').text

#     # if parti in ['ELDR', 'GUE-NGL', 'PPE-DE', 'PSE', 'Verts-ALE']:  # Limiter aux principaux partis
#     data.append([doc_id, "Anonyme", texte])

# fichier_csv = 'test_corpus_fr.csv'
# with open(fichier_csv, mode='w', newline='', encoding='utf-8') as file:
#     writer = csv.writer(file)
#     writer.writerow(['ID', 'Intervenant', 'Texte', 'Parti Politique'])
#     writer.writerows(data)



In [144]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import pandas as pd
tokenizer = RegexpTokenizer(r'\w+')


custom_stop_words = set(stopwords.words('french'))
custom_stop_words = set(stopwords.words('french') + [
    'monsieur', 'madame', 'président', 'présidente', 
    'commissaire', 'groupe', 'rapport', 'parlement'
])

# Prétraitement avancé
import re
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import spacy

# Charger le modèle français de SpaCy
# nlp = spacy.load("fr_core_news_md")
nlp = spacy.load("fr_core_news_sm")


# Liste de stopwords personnalisée
custom_stop_words = set(stopwords.words('french') + [
    'monsieur', 'madame', 'président', 'présidente', 
    'commissaire', 'groupe', 'rapport', 'parlement'
])

def preprocess_text(text):
    """
    Prétraitement avancé du texte : 
    - Minuscules
    - Suppression des ponctuations et caractères spéciaux
    - Suppression des stopwords
    - Lemmatisation
    """
    # Convertir en minuscules
    text = text.lower()
    
    # Supprimer les balises HTML et caractères spéciaux
    text = re.sub(r'<[^>]+>', ' ', text)  # Supprime les balises HTML
    text = re.sub(r'[^\w\s]', ' ', text)  # Supprime les caractères spéciaux
    
    # Tokenization avec SpaCy
    doc = nlp(text)
    
    # Lemmatisation, suppression des mots vides et des mots très courts
    tokens = [
        token.lemma_ for token in doc 
        if token.is_alpha and  # Garder uniquement les mots (pas de chiffres)
        len(token) > 2 and     # Supprimer les mots très courts
        token.lemma_ not in custom_stop_words
    ]
    
    # Joindre les tokens pour reconstruire le texte
    processed_text = " ".join(tokens)
    return processed_text


# Chargement et prétraitement des données
def load_and_preprocess_data(train_path, test_path):
    # Données d'entraînement
    train_data = pd.read_csv(train_path)
    train_data['Texte'] = train_data['Texte'].fillna("")
    # .apply(preprocess_text)
    
    # Données de test
    test_data = pd.read_csv(test_path)
    test_data['Texte'] = test_data['Texte'].fillna("")
    # .apply(preprocess_text)
    
    return train_data, test_data

# Charger les données
train_df, test_df = load_and_preprocess_data("corpus_fr.csv", "test_corpus_fr.csv")

# Préparation pour le machine learning
X_data_train = train_df['Texte']
y_data_train = train_df['Parti Politique']
data_test = test_df['Texte']





with open("data/deft09_parlement_ref_fr.txt", "r", encoding="utf-8") as f:
    test_labels = [line.strip().split("\t") for line in f.readlines()]  # Adapter selon le format du fichier

# Convertir les labels en DataFrame
df_test_labels = pd.DataFrame(test_labels, columns=["id", "Parti Politique"])

merge_data_test =  pd.concat([data_test, df_test_labels], axis=1)
merge_data_test = merge_data_test[["Texte","Parti Politique"]]
merge_data_test = merge_data_test.dropna()

X_data_test = merge_data_test['Texte']
y_data_test = merge_data_test['Parti Politique']  

In [145]:
from sklearn.utils import resample
import pandas as pd

# Trouver la taille minimale
min_size = train_df['Parti Politique'].value_counts().min()

# Grouper par classe et sous-échantillonner
balanced_train = pd.concat([
    resample(group, replace=False, n_samples=min_size, random_state=42)
    for _, group in train_df.groupby('Parti Politique')
])

print(balanced_train['Parti Politique'].value_counts())


Parti Politique
ELDR         2008
GUE-NGL      2008
PPE-DE       2008
PSE          2008
Verts-ALE    2008
Name: count, dtype: int64


In [146]:
from sklearn.utils import resample
import pandas as pd

# Trouver la taille minimale
min_size = merge_data_test['Parti Politique'].value_counts().min()

# Grouper par classe et sous-échantillonner
balanced_test = pd.concat([
    resample(group, replace=False, n_samples=min_size, random_state=42)
    for _, group in train_df.groupby('Parti Politique')
])
print(balanced_test['Parti Politique'].value_counts())


Parti Politique
ELDR         1339
GUE-NGL      1339
PPE-DE       1339
PSE          1339
Verts-ALE    1339
Name: count, dtype: int64


In [147]:
# X_data_train = balanced_train['Texte']
# y_data_train = balanced_train['Parti Politique']

# X_data_test = balanced_test['Texte']
# y_data_test = balanced_test['Parti Politique'] 
# X_data_test

In [148]:
len(X_data_train)

19370

In [149]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords



# Vectorisation TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_data_train)
X_test_tfidf = tfidf_vectorizer.transform(X_data_test)

# Entraîner un modèle SVM
svm_classifier = SVC(kernel='rbf', class_weight='balanced', random_state=42)

svm_classifier.fit(X_train_tfidf, y_data_train)

# Prédictions sur le jeu de test
y_pred = svm_classifier.predict(X_test_tfidf)

# Évaluation des performances
accuracy = accuracy_score(y_data_test, y_pred)
print("Accuracy :", accuracy)
print("\nRapport de classification :\n", classification_report(y_data_test, y_pred))

Accuracy : 0.7279907084785133

Rapport de classification :
               precision    recall  f1-score   support

        ELDR       0.78      0.67      0.72      1339
     GUE-NGL       0.70      0.80      0.75      1793
      PPE-DE       0.75      0.74      0.74      4571
         PSE       0.69      0.72      0.71      3627
   Verts-ALE       0.73      0.68      0.71      1585

    accuracy                           0.73     12915
   macro avg       0.73      0.72      0.73     12915
weighted avg       0.73      0.73      0.73     12915



# 3. Représentation par espaces de thèmes

In [150]:
french_stop_words = stopwords.words('french')  # Vérifiez l'importation



In [151]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Vectorisation des textes
vectorizer = CountVectorizer(
    max_df=0.95, min_df=2, stop_words=french_stop_words
)
X_train_counts = vectorizer.fit_transform(X_data_train)
X_test_counts = vectorizer.transform(X_data_test)

# Entraîner LDA
lda_model = LatentDirichletAllocation(
    n_components=5,  # Nombre de thèmes
    doc_topic_prior=1.0,  # Prior des thèmes dans chaque document
    topic_word_prior=0.8,  # Prior des mots dans chaque thème
    random_state=42
)

# Apprentissage sur les données d'entraînement
X_train_lda = lda_model.fit_transform(X_train_counts)

# Projection des données de test dans l'espace des thèmes
X_test_lda = lda_model.transform(X_test_counts)

# Afficher les thèmes
feature_names = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda_model.components_):
    print(f"Thème {topic_idx}:")
    print(" ".join([feature_names[i] for i in topic.argsort()[:-11:-1]]))

# Entraîner un classifieur SVM avec les représentations thématiques
svm_classifier = SVC(kernel='rbf', class_weight='balanced', random_state=42)
svm_classifier.fit(X_train_lda, y_data_train)

# Prédictions et évaluation
y_pred = svm_classifier.predict(X_test_lda)
accuracy = accuracy_score(y_data_test, y_pred)
print(f"Accuracy: {accuracy}")
print("Rapport de classification:")
print(classification_report(y_data_test, y_pred))


Thème 0:
président monsieur cette question contre fait présidente si plus être
Thème 1:
pays union européenne président monsieur politique plus europe droits états
Thème 2:
commission président monsieur parlement rapport voudrais cette conseil tout madame
Thème 3:
plus être européenne monsieur états cette président marché membres comme
Thème 4:
monsieur président commissaire rapport messieurs mesdames plus madame collègues fait
Accuracy: 0.24359272164150214
Rapport de classification:
              precision    recall  f1-score   support

        ELDR       0.11      0.14      0.12      1339
     GUE-NGL       0.25      0.57      0.35      1793
      PPE-DE       0.42      0.06      0.10      4571
         PSE       0.34      0.35      0.35      3627
   Verts-ALE       0.14      0.24      0.17      1585

    accuracy                           0.24     12915
   macro avg       0.25      0.27      0.22     12915
weighted avg       0.31      0.24      0.22     12915

